In [33]:
import torch 
import os
from utils.seqlab import preprocessing_kmer
from transformers import BertTokenizer
from tqdm import tqdm

device = "cuda:0"
tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
test_file = os.path.join("workspace", "seqlab-latest", "test.csv")
batch_size = 8
test_dataloader = preprocessing_kmer(test_file, tokenizer, batch_size)
test_size = len(test_dataloader)


Preparing Data test: 100%|██████████| 6961/6961 [00:55<00:00, 125.73it/s]


In [10]:
# somehow I do PCA analysis.
from models import seqlab, pretrained
from transformers import BertForMaskedLM
import os
import json
import torch

m = seqlab.DNABERT_SL(
    BertForMaskedLM.from_pretrained(os.path.join("pretrained", "3-new-12w-0")).bert,
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)
n = pretrained.DNABERT_SL.from_pretrained(
    os.path.join("pretrained", "3-new-12w-0"),
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)

checkpoint = torch.load(
    os.path.join("run", "sso01-adamw-lr5e-5-base.lin1-2w1boplw", "latest", "checkpoint.pth"), 
    map_location="cuda:0") # force to cuda:0 device
m.load_state_dict(checkpoint.get("model"))
n.load_state_dict(checkpoint.get("model"))

Some weights of the model checkpoint at pretrained\3-new-12w-0 were not used when initializing DNABERT_SL: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing DNABERT_SL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DNABERT_SL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DNABERT_SL were not initialized from the model checkpoint at pretrained\3-new-12w-0 and are newly initial

<All keys matched successfully>

In [29]:
n.save_pretrained(os.path.join("pretrained", "dnabert-sl-lin1"))

In [27]:
m.train()
n.train()
str(m.state_dict()) == str(n.state_dict())

True

In [34]:
import torch
from utils.seqlab import id2label
import pandas as pd

m.eval()
# n.eval()
m.to(device)
# n.to(device)
token_ids, target_ids, bert_outputs = [], [], []
for step, batch in tqdm(enumerate(test_dataloader), total=test_size, desc="Testing"):
    input_ids, attn_mask, token_type_ids, target_labels = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        out, out_bert, out_head = m(input_ids, attn_mask)
        for b_input_ids, b_target, b_bert in zip(input_ids, target_labels, out_bert):
            for i, j, k in zip(b_input_ids, b_target, b_bert):
                token_ids.append(i.item())
                target_ids.append(j.item())
                bert_outputs.append(" ".join([str(a) for a in k.tolist()]))

df = pd.DataFrame(data={
    "token_id": token_ids,
    "target_id": target_ids,
    "bert_value": bert_outputs
})
df.head(5)

Testing: 100%|██████████| 871/871 [1:28:01<00:00,  6.06s/it]   


,token_id,target_id,bert_value
0,2,-100,1.148287057876587 1.622141718864441 -1.0630100...
1,29,7,0.2518802285194397 1.3096948862075806 0.932622...
2,38,7,0.2287495732307434 1.3885688781738281 0.312197...
3,10,7,0.41064363718032837 1.1078848838806152 0.12063...
4,28,7,0.8710600137710571 1.133277416229248 0.5390868...


In [35]:
df.to_csv(os.path.join("motif_analysis", "token_analysis", "token_bert_value.csv"), index=False)

In [9]:
n

DNABERT_SL(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(69, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [2]:
# get token prediction from categorized dataframes.
import os
import pandas as pd
path = os.path.join("prediction", "dataframe-0.9=F1 Score-1.0.csv")
df = pd.read_csv(path)
df.head(5)

,label_sequence_id,input_tokens,prediction_ids,target_ids,step,f1_score-iii,precision-iii,recall-iii,f1_score-iiE,precision-iiE,...,prediction_tokens,target_tokens,prediction,target,nucl_precision_intron,nucl_precision_exon,nucl_recall_intron,nucl_recall_exon,nucl_f1_intron,nucl_f1_exon
0,491,ATG TGA GAA AAT ATC TCC CCA CAC ACT CTC TCA CA...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,1301,0.924,0.858,1.0,1.0,1.0,...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...,0.860,1.0,1.0,0.430,0.925,0.601
1,515,CCA CAC ACT CTC TCA CAA AAA AAT ATG TGT GTT TT...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,1355,0.926,0.862,1.0,1.0,1.0,...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiii...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiii...,0.863,1.0,1.0,0.407,0.926,0.579
2,517,GCC CCA CAT ATC TCC CCC CCA CAG AGC GCC CCA CA...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,1357,0.997,0.994,1.0,1.0,1.0,...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiii...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiii...,0.989,1.0,1.0,0.898,0.994,0.946
3,561,GCC CCA CAT ATG TGA GAC ACA CAA AAC ACA CAG AG...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 ...,1451,0.986,0.973,1.0,1.0,1.0,...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EEE EE...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiiiiiiiiiiii...,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEiiiiiiiiiiiiiii...,0.969,1.0,1.0,0.694,0.984,0.819
4,670,ACA CAT ATT TTA TAT ATC TCA CAG AGG GGT GTA TA...,7 7 7 7 7 7 7 7 5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,7 7 7 7 7 7 7 7 5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1702,0.998,0.996,1.0,1.0,1.0,...,EEE EEE EEE EEE EEE EEE EEE EEE EEi Eii iii ii...,EEE EEE EEE EEE EEE EEE EEE EEE EEi Eii iii ii...,EEEEEEEEEEiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii...,EEEEEEEEEEiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii...,0.992,1.0,1.0,0.714,0.996,0.833


In [3]:
from transformers import BertTokenizer, BertForMaskedLM
from utils.seqlab import _process_sequence_and_label, id2label, label2id
from models import seqlab
import json
from tqdm import tqdm
from torch import tensor, no_grad, argmax

tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
model = seqlab.DNABERT_SL(
    BertForMaskedLM.from_pretrained(os.path.join("pretrained", "3-new-12w-0")).bert,
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)
model.eval()
device = "cuda:0"
model.to(device)
ss_labels = [
    "iiE",
    "iEE", 
    "EEi", 
    "Eii"
]
ss_label_ids = [label2id[a] for a in ss_labels]
arr_tokens, arr_labels, arr_bert_values, arr_pred_tokens = [], [], [], []
for i, r in tqdm(df.iterrows(), total=df.shape[0], desc="Testing"):
    with no_grad():
        b_input_ids, b_attention_mask, b_token_type_ids, b_label_ids = _process_sequence_and_label(
            r["input_tokens"],
            r["target_tokens"],
            tokenizer
        )
        b_input_ids = tensor([b_input_ids]).to(device)
        b_attention_mask = tensor([b_attention_mask]).to(device)
        b_label_ids = tensor([b_label_ids]).to(device)
    b_out, b_out_bert, b_out_head = model(
        b_input_ids, 
        b_attention_mask
    )
    b_pred_ids = argmax(b_out)
    # print(b_input_ids.shape, b_label_ids.shape, b_out_bert.shape)
    for input_ids, label_ids, out_bert, pred_ids in zip(b_input_ids, b_label_ids, b_out_bert, b_pred_ids):
        for i, j, k, p in zip(input_ids, label_ids, out_bert, pred_ids):
            label_id = j.item()
            if label_id >= 0:
                if label_id in ss_label_ids:
                    token_label = id2label[label_id]
                    token = tokenizer.ids_to_tokens[i.item()]
                    pred_token= id2label[p.item()]
                    bert_values_str = [str(a) for a in k.tolist()]
                    arr_tokens.append(token)
                    arr_pred_tokens.append(pred_token)
                    arr_labels.append(token_label)
                    arr_bert_values.append(" ".join(bert_values_str))

ndf = pd.DataFrame(data={
    "token": arr_tokens,
    "label": arr_labels,
    "bert_value": arr_bert_values
})
ndf.head(5)


Testing:   0%|          | 0/210 [00:05<?, ?it/s]


NameError: name 'torch' is not defined

In [38]:
ndf = pd.DataFrame(data={
    "token": arr_tokens,
    "label": arr_labels,
    "bert_value": arr_bert_values
})
ndf.head(5)
ndf.to_csv(os.path.join("motif_analysis", "token_analysis", "df_0.9=F1-1.csv"), index=False)